In [3]:
from openai import OpenAI
import os
import logging
from jinja2 import Environment, select_autoescape
import json
from miniautogen.pipeline.pipeline import PipelineComponent
from miniautogen.llms.llm_client import OpenAIClient, LiteLLMClient
from dotenv import load_dotenv
load_dotenv()

api_key=os.getenv('OPENAI_API_KEY')
openai_client = OpenAIClient(api_key=api_key)
litellm_client = LiteLLMClient(model="ollama/phi")

In [5]:
from miniautogen.chat.chat import Chat
from miniautogen.agent.agent import Agent
from miniautogen.chat.chatadmin import ChatAdmin
from miniautogen.pipeline.pipeline import Pipeline
from miniautogen.pipeline.components.components import (
    UserResponseComponent,
    AgentReplyComponent,
    TerminateChatComponent,
    Jinja2SingleTemplateComponent,
    UpdateNextAgentComponent,
    NextAgentMessageComponent,
    LLMResponseComponent
)

template_str = """
[
  {"role": "system", "content": "{{ agent.role }}"}{% for message in messages %},
  {% if message.sender_id == agent.agent_id %}
    {"role": "assistant", "content": {{ message.message | tojson | safe }}}
  {% else %}
    {"role": "user", "content": {{ message.message | tojson | safe }}}
  {% endif %}
{% endfor %}
]
"""

# Cria uma instância do componente com o template
jinja_component = Jinja2SingleTemplateComponent()
jinja_component.set_template_str(template_str)

UpdateNextAgent = UpdateNextAgentComponent()
UpdateNextAgent.set_next_agent_id("Bruno")

NextAgentMessage = NextAgentMessageComponent()
NextAgentMessage.set_alternative_next(UpdateNextAgent)

# Configuração dos Pipelines
pipeline_user = Pipeline([UserResponseComponent()])
pipeline_jinja = Pipeline([jinja_component, LLMResponseComponent(litellm_client)])
pipeline_admin = Pipeline(
    [NextAgentMessage, AgentReplyComponent(), TerminateChatComponent()])

# Setup do ambiente de teste
chat = Chat()
json_data = {
    'agent_id': 'Bruno',
    'name': 'Bruno',
    'role': 'user'
}

# Criação de Agentes
agent1 = Agent.from_json(json_data)
agent1.pipeline = pipeline_user  # Atribuindo o pipeline ao agente

agent2 = Agent("dev", "Carlos", "Python senior Developer")
agent2.pipeline = pipeline_jinja  # Atribuindo o pipeline_llm ao segundo agente


# Adicionando os agentes ao chat
chat.add_agent(agent1)
chat.add_agent(agent2)

# Adição de mensagens de teste ao chat
json_messages = [
    {'sender_id': 'Bruno', 'message': 'Its a test, dont worry'},
]
chat.add_messages(json_messages)

# Criação e configuração do ChatAdmin
chat_admin = ChatAdmin("admin", "Admin", "admin_role",
                       pipeline_admin, chat, 10)

In [6]:
chat_admin.run()

INFO:miniautogen.chat.chatadmin:Chat Admin started.
INFO:miniautogen.chat.chatadmin:Executing round 1


No matching agent found in the last message.
Next agent updated to: Bruno


INFO:miniautogen.chat.chatadmin:Executing round 2


Teste
No matching agent found in the last message.
Next agent updated to: Bruno


INFO:miniautogen.chat.chatadmin:Executing round 3


dev
[{'role': 'system', 'content': 'Python senior Developer'}, {'role': 'user', 'content': "\nOn a spring afternoon in Paris, the sun poured its golden light over the cobblestone streets of the city, illuminating the scene like an Impressionist painting coming to life. At the heart of the French capital lay Café Parisiense, a picturesque refuge that seemed to have sprung from a 19th-century romance.\n\nThe café's tables sprawled beneath the canopy of flowering trees, their green leaves swaying gently to the rhythm of a delicate breeze. Each table was adorned with fresh bouquets of colorful flowers, and the soft sound of a violin in the background filled the air with melodic notes, as if Paris were composing its own romantic soundtrack.\n\nIn this idyllic setting, three destinies intertwined on that day. The first character, Alex, an exceptionally talented local artist, sat at one of the café's tables, his gaze fixed on the sketchbook before him. He frequented Café Parisiense regularly,

INFO:miniautogen.chat.chatadmin:Executing round 4




As Alex continued to capture the beauty of Camille, he couldn't help but feel a sense of familiarity in her presence. The way she carried herself reminded him of his childhood friend, who had left years ago to pursue her dreams in Paris. He wondered if their paths would cross again someday and what their story might be.
Meanwhile, Jordan was enjoying the vibrant atmosphere of Café Parisiense. The music, the art, and the people all seemed to blend together perfectly, creating a magical experience for him. As he sipped his coffee and observed the surroundings, he noticed Alex working on a sketch and couldn't help but admire the talent behind it.
Little did they know that their paths were about to cross in a way that would change everything. The café was bustling with life, filled with laughter, love, and dreams coming true. And in this moment, three destinies intertwined, setting the stage for an unforgettable romance.

No matching agent found in the last message.
Next agent updated to

KeyboardInterrupt: Interrupted by user

In [ ]:
chat.get_messages().to_dict(orient='records')

[{'id': 1,
  'sender_id': 'Bruno',
  'message': 'Its a test, dont worry',
  'timestamp': Timestamp('2024-01-02 21:22:06.723001'),
  'additional_info': None},
 {'id': 2,
  'sender_id': 'Bruno',
  'message': 'dev, Can you develop in python?',
  'timestamp': Timestamp('2024-01-02 21:22:24.434003'),
  'additional_info': None},
 {'id': 3,
  'sender_id': 'dev',
  'message': ' Sure, I can help you with Python. What do you need assistance with?\n',
  'timestamp': Timestamp('2024-01-02 21:22:28.978886'),
  'additional_info': None},
 {'id': 4,
  'sender_id': 'Bruno',
  'message': 'Can you develop a api?',
  'timestamp': Timestamp('2024-01-02 21:23:09.165525'),
  'additional_info': None},
 {'id': 5,
  'sender_id': 'dev',
  'message': ' Sure, I can help you develop an API. What type of API are you looking to create?\n',
  'timestamp': Timestamp('2024-01-02 21:23:11.509733'),
  'additional_info': None},
 {'id': 6,
  'sender_id': 'dev',
  'message': ' Sure, I can help you develop an API. What type o